# Wetland Mapping with GeoAI

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/wetland_mapping.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [4]:
# %pip install geoai-py

## Import libraries

In [2]:
import leafmap
import geoai

ModuleNotFoundError: No module named 'leafmap'

## Download sample NAIP imagery

Sample NAIP imagery: https://huggingface.co/datasets/giswqs/geospatial/tree/main/naip

In [3]:
train_raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip/m_4609931_ne_14_1_20100629.tif"
test_raster_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip/m_4609932_nw_14_1_20100629.tif"

In [ ]:
train_raster_path = geoai.download_file(train_raster_url)
test_raster_path = geoai.download_file(test_raster_url)

In [ ]:
geoai.print_raster_info(test_raster_path, show_preview=False)

In [ ]:
geoai.view_raster(train_raster_url)

In [ ]:
geoai.view_raster(test_raster_url)

## Download sample wetland data

National Wetlands Inventory: https://www.fws.gov/program/national-wetlands-inventory/wetlands-mapper

In [ ]:
m = leafmap.Map()
m.add_cog_layer(train_raster_url, bidx=[4, 1, 2], name="NAIP Imagery")
m

In [ ]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [-99.192, 46.56, -99.1206, 46.6283]

In [ ]:
bbox_geometry = leafmap.bbox_to_gdf(bbox)

In [ ]:
nwi_gdf = leafmap.get_nwi(bbox_geometry)
nwi_gdf.head()

In [ ]:
m.add_nwi(nwi_gdf, col_name="Wetlands.WETLAND_TYPE")

In [ ]:
nwi_geojson = "nwi_wetlands.geojson"
nwi_gdf.to_file(nwi_geojson)

## Create training data

In [ ]:
out_folder = "output"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=nwi_geojson,
    tile_size=512,
    stride=256,
    buffer_radius=0,
)

## Train object detection model

In [ ]:
geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models",
    num_channels=4,
    pretrained=True,
    batch_size=4,
    num_epochs=10,
    learning_rate=0.005,
    val_split=0.2,
)

## Run inference

In [ ]:
masks_path = "naip_test_prediction.tif"
model_path = f"{out_folder}/models/best_model.pth"

In [ ]:
geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=512,
    overlap=256,
    confidence_threshold=0.3,
    batch_size=4,
    num_channels=4,
)

## Vectorize masks

In [ ]:
output_path = "naip_test_prediction.geojson"
gdf = geoai.raster_to_vector(
    masks_path, output_path, min_area=1000, simplify_tolerance=1
)

## Visualize results

In [ ]:
geoai.view_vector_interactive(output_path, tiles=test_raster_url)

In [ ]:
geoai.create_split_map(
    left_layer=output_path,
    right_layer=test_raster_url,
    left_args={"style": {"color": "red", "fillOpacity": 0.4}},
    basemap=test_raster_url,
)

![image](https://github.com/user-attachments/assets/ab17cb29-7426-4090-8d02-b3b98dad5d05)